In [12]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import requests
import random
from datetime import datetime

load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')
NEWS_API_KEY = os.getenv('NEWS_API_KEY')

CHAT_MODEL = 'gpt-4o-mini'

def abbreviate(text, length=6):
    if len(text) <= length:
        return text
    
    return text[:length] + "**"

print('ready', abbreviate(OPENAI_KEY), abbreviate(NEWS_API_KEY), CHAT_MODEL)

ready sk-pro** 145405** gpt-4o-mini


In [13]:
class PostGenerator:


    def __init__(self, chat_model=CHAT_MODEL):
        client = OpenAI(
            api_key=OPENAI_KEY,  # this is also the default, it can be omitted
        )
        self.client = client
        self.chat_model = chat_model
        self.max_tokens = 160

    # https://platform.openai.com/docs/guides/text-generation#text-generation-models
    def create_response_for_messages(self, messages):
        response = self.client.chat.completions.create(
            model=self.chat_model,
            messages=messages,
            max_tokens=self.max_tokens
        )
        return response.choices[0].message.content


    def generate_tweet(self, user_background, target_audience, event):
        prompt = f"""Generate a formatted tweet (with newlines) for someone with a background in {user_background}, targeting {target_audience}. Current event: {event}.
        -- Using this event for the post take a hard stance or side on the topic with a short explanation as to why. No emojis or hashtags.
        Compare it to something in the past"""
        messages = [
            {'role': 'developer', 'content': 'You are a social media manager responsible for making posts. Do not make these posts sound robotic. Insert abbreviations and language to make the post sound human. Do not mention the target audience but use it to influence the writing.'},
            {'role': 'user', 'content': prompt}
        ]
        tweet = self.create_response_for_messages(messages)
        return tweet

    def rate_tweet(self, tweet):
        prompt = f"Rate the following tweet on its effectiveness from 1-10 and provide the top 3 suggestions for improvement:\n\n{tweet}"
        messages = [
            {'role': 'user', 'content': prompt}
        ]
        response = self.create_response_for_messages(messages)
        return response

    def improve_tweet(self, tweet, suggestions):
        prompt = f"Improve the following tweet based on these suggestions:\n\nTweet: {tweet}\n\nSuggestions: {suggestions}. The post should not have emojis or hashtags."
        messages = [
            {'role': 'user', 'content': prompt}
        ]
        response = self.create_response_for_messages(messages)
        return response

    def generate_image_from_post(self, post):
        prompt = f"""Generate an interesting/funny social media image based on this post snippet: {post[:100]}"""
        response = self.client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            n=1,
            size="1024x1024"
        )
        return response

    def save_tweet_and_image(self, tweet, image_response=None):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_dir = os.path.join("output", timestamp)
        os.makedirs(output_dir, exist_ok=True)

        tweet_path = os.path.join(output_dir, "tweet.txt")
        with open(tweet_path, "w") as f:
            f.write(tweet)

        if image_response:
            for i, img in enumerate(image_response.data):
                image_path = os.path.join(output_dir, f"image_{i}.png")
                with open(image_path, "wb") as f:
                    f.write(requests.get(img.url).content)


In [14]:
def format_article(a):
    text = f"{a['title']} {a["description"]}"
    return text.replace('[Removed]', '')

def get_current_events(theme):
    response = requests.get('https://newsapi.org/v2/everything', params={
        'apiKey': NEWS_API_KEY,
        'q': theme,
        'pageSize': 5
    })
    articles = response.json()
    articles = articles.get('articles', [])
    articles = [format_article(a) for a in articles]
    return articles

current_events = None

In [15]:
# Example usage
background = "software, engineering, science, entrepreneurship"
interests = "coding, open-source projects, hackathons, and startups"
target_audience = "developers, entrepreneurs"
theme = "technology, science, controversial"

if not current_events:
  current_events = get_current_events(theme)
  print('fetched events', current_events)
else:
  print('using cached events', current_events)

fetched events ['Biden backs calls to ban congressional stock trading President Joe Biden lent his voice to the growing bipartisan calls on the Hill to end stock trading for congressional members.', 'The Download: words of wisdom from the departing White House tech advisor, and controversial AI manga translation This is today’s edition of\xa0The Download,\xa0our weekday newsletter that provides a daily dose of what’s going on in the world of technology. What the departing White House chief tech advisor has to say on AI President Biden’s administration will end within two m…', 'House Passes Bill That Bans Transgender Care for Military Members’ Children The House approved a $895 billion bill, which included a provision prohibiting gender-affirming care for transgender children of service members', 'Study likely to change standard of care for deadly strokes Endovascular therapy, a minimally invasive surgery performed inside the blood vessels, is preferred to alternative approaches for ves

In [16]:
random_index = 0 if len(current_events) == 1 else random.randint(0, len(current_events) - 1)

# use random index or select

event = current_events[1]

post_generator = PostGenerator()



In [ ]:
random_index = 0 if len(current_events) == 1 else random.randint(0, len(current_events) - 1)

event = current_events[random_index]
print('selected event', event)

generator = PostGenerator()
tweet = generator.generate_tweet(background, target_audience, event)
print('generated tweet')
print(tweet)

try:
  image_response = generator.generate_image_from_post(tweet)
  print(image_response)
except Exception as e:
  print('error generating image', e)
  image_response = None

generator.save_tweet_and_image(tweet, image_response)


selected event Biden backs calls to ban congressional stock trading President Joe Biden lent his voice to the growing bipartisan calls on the Hill to end stock trading for congressional members.
generated tweet
Big news: Biden’s backing a push to ban congressional stock trading. 

This isn't just about ethics; it’s about transparency in our governing bodies. 

Remember the Enron scandal? We learned 1st hand how unchecked power can lead to corruption. 

If our leaders can profit from insider info, how can we trust their decisions? Time for action!
